In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# 이미지 데이터 경로 설정
data_dir = 'fer2013_train' # 폴더 경로 설정 (사이즈 작은 이미지 / 2,100장 테스트)
input_shape = (224, 224)
batch_size = 32
num_classes = 7  # 클래스 개수에 맞게 설정

# 훈련셋, 검증셋, 테스트셋으로 데이터 나누기

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.3,  # 훈련셋 70%, 검증셋 30%
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# 훈련셋, 검증셋, 테스트셋으로 데이터 나누기
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  # 훈련셋 80%, 검증셋 20%
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# 훈련셋 제너레이터 생성
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # 'training' subset은 훈련셋으로 사용
)

# 검증셋 제너레이터 생성
valid_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # 'validation' subset은 검증셋으로 사용
)

# 테스트셋 제너레이터 생성
test_dir = 'fer2013_test'
test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical',
    subset=None 
)


# 모델 체크포인트 콜백 정의
checkpoint_path = 'DenseNet_testcode_V7_2100test_01.h5'
checkpoint = ModelCheckpoint(
    checkpoint_path, 
    monitor='val_accuracy',  # 저장할 모델을 선택할 지표 설정 (여기서는 검증 정확도)
    save_best_only=True,     # 최상의 성능을 보이는 경우에만 저장
    mode='max',              # 지표의 최대값을 찾음
    verbose=1                # 저장할 때 메시지 출력
)

Found 22968 images belonging to 1 classes.
Found 5741 images belonging to 1 classes.
Found 7178 images belonging to 1 classes.


In [3]:
def DenseNet(x):
    # input = 224 x 224 x 3
    k = 32  # Grow Rate
    compression = 0.2   # 밀집 블록 내에서 출력 채널 수를 줄임 (모델의 파라미터 수와 연산량을 줄이면서도 효과적인 특성 추출)


    # 1. Convolution
    # 입력 이미지에 7x7 크기의 필터를 사용하는 Convolution 레이어를 적용
    x = layers.Conv2D(k * 2, (7, 7), strides=2, padding='same', input_shape=(224, 224, 3))(x)    # 112x112x64
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # 2. Pooling
    # 이미지 내의 중요한 특성을 보존하면서 크기를 줄임
    x = layers.MaxPool2D((3, 3), 2, padding='same')(x)  # 56x56x64


    # 5. Dense Block (2)
    for i in range(3) :
        x_l = layers.Conv2D(k * 4, (1, 1), strides=1, padding='same')(x)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x_l = layers.Conv2D(k, (3, 3), strides=1, padding='same')(x_l)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x = layers.Concatenate()([x, x_l])


    # 6. Transition Layer (2)
    current_shape = int(x.shape[-1])
    x = layers.Conv2D(int(current_shape * compression), (1, 1), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.AveragePooling2D((2, 2), strides=2, padding='same')(x)   # 14x14


    # 7. Dense Block (3)
    for i in range(24) :
        x_l = layers.Conv2D(k * 4, (1, 1), strides=1, padding='same')(x)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x_l = layers.Conv2D(k, (3, 3), strides=1, padding='same')(x_l)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x = layers.Concatenate()([x, x_l])

    # 8. Transition Layer (3)
    current_shape = int(x.shape[-1])
    x = layers.Conv2D(int(current_shape * compression), (1, 1), strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.AveragePooling2D((2, 2), strides=2, padding='same')(x)   # 7x7


    # 9. Dense Block (4)
    for i in range(6) :
        x_l = layers.Conv2D(k * 4, (1, 1), strides=1, padding='same')(x)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x_l = layers.Conv2D(k, (3, 3), strides=1, padding='same')(x_l)
        x_l = layers.BatchNormalization()(x_l)
        x_l = layers.Activation('relu')(x_l)

        x = layers.Concatenate()([x, x_l])

    # 10. Classification Layer
    x = layers.GlobalAveragePooling2D()(x)
    # classes = 7 (softmax)
    x = layers.Dense(7, activation='softmax')(x)

    return x

In [4]:
# 모델 생성
input_layer = layers.Input(shape=(*input_shape, 3), dtype='float32', name='input')
output_layer = DenseNet(input_layer)
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [5]:
# 클래스 개수 설정
num_classes = 7

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='categorical_crossentropy',  # 다중 클래스 분류용 함수
              metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['input[0][0]']               
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 112, 112, 64)         0         ['batch_normalization[0][0

 conv2d_8 (Conv2D)           (None, 28, 28, 128)          4224      ['average_pooling2d[0][0]']   
                                                                                                  
 batch_normalization_8 (Bat  (None, 28, 28, 128)          512       ['conv2d_8[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_8 (Activation)   (None, 28, 28, 128)          0         ['batch_normalization_8[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 28, 28, 32)           36896     ['activation_8[0][0]']        
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_17 (Conv2D)          (None, 28, 28, 32)           36896     ['activation_16[0][0]']       
                                                                                                  
 batch_normalization_17 (Ba  (None, 28, 28, 32)           128       ['conv2d_17[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_17 (Activation)  (None, 28, 28, 32)           0         ['batch_normalization_17[0][0]
                                                                    ']                            
                                                                                                  
 concatenate_7 (Concatenate  (None, 28, 28, 192)          0         ['concatenate_6[0][0]',       
 )        

                                                                    ']                            
                                                                                                  
 concatenate_11 (Concatenat  (None, 28, 28, 320)          0         ['concatenate_10[0][0]',      
 e)                                                                  'activation_25[0][0]']       
                                                                                                  
 conv2d_26 (Conv2D)          (None, 28, 28, 128)          41088     ['concatenate_11[0][0]']      
                                                                                                  
 batch_normalization_26 (Ba  (None, 28, 28, 128)          512       ['conv2d_26[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activatio

 batch_normalization_34 (Ba  (None, 28, 28, 128)          512       ['conv2d_34[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_34 (Activation)  (None, 28, 28, 128)          0         ['batch_normalization_34[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_35 (Conv2D)          (None, 28, 28, 32)           36896     ['activation_34[0][0]']       
                                                                                                  
 batch_normalization_35 (Ba  (None, 28, 28, 32)           128       ['conv2d_35[0][0]']           
 tchNormalization)                                                                                
          

                                                                                                  
 batch_normalization_43 (Ba  (None, 28, 28, 32)           128       ['conv2d_43[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_43 (Activation)  (None, 28, 28, 32)           0         ['batch_normalization_43[0][0]
                                                                    ']                            
                                                                                                  
 concatenate_20 (Concatenat  (None, 28, 28, 608)          0         ['concatenate_19[0][0]',      
 e)                                                                  'activation_43[0][0]']       
                                                                                                  
 conv2d_44

 concatenate_24 (Concatenat  (None, 28, 28, 736)          0         ['concatenate_23[0][0]',      
 e)                                                                  'activation_51[0][0]']       
                                                                                                  
 conv2d_52 (Conv2D)          (None, 28, 28, 128)          94336     ['concatenate_24[0][0]']      
                                                                                                  
 batch_normalization_52 (Ba  (None, 28, 28, 128)          512       ['conv2d_52[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_52 (Activation)  (None, 28, 28, 128)          0         ['batch_normalization_52[0][0]
                                                                    ']                            
          

                                                                                                  
 activation_60 (Activation)  (None, 14, 14, 32)           0         ['batch_normalization_60[0][0]
                                                                    ']                            
                                                                                                  
 concatenate_28 (Concatenat  (None, 14, 14, 224)          0         ['concatenate_27[0][0]',      
 e)                                                                  'activation_60[0][0]']       
                                                                                                  
 conv2d_61 (Conv2D)          (None, 14, 14, 128)          28800     ['concatenate_28[0][0]']      
                                                                                                  
 batch_normalization_61 (Ba  (None, 14, 14, 128)          512       ['conv2d_61[0][0]']           
 tchNormal

 global_average_pooling2d (  (None, 352)                  0         ['concatenate_32[0][0]']      
 GlobalAveragePooling2D)                                                                          
                                                                                                  
 dense (Dense)               (None, 7)                    2471      ['global_average_pooling2d[0][
                                                                    0]']                          
                                                                                                  
Total params: 2839175 (10.83 MB)
Trainable params: 2828103 (10.79 MB)
Non-trainable params: 11072 (43.25 KB)
__________________________________________________________________________________________________


In [6]:
# model = tf.keras.models.load_model(checkpoint_path) # checkpoint 모델 불러오기

In [ ]:
# 모델 학습
epochs = 50
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size
)

# 훈련 과정의 손실과 정확도 추출
train_loss = history.history['loss']
train_acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

# 테스트셋 평가
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)

# 결과 출력
print("Training Loss:", train_loss)
print("Training Accuracy:", train_acc)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/50
400/718 [===============>..............] - ETA: 29:12 - loss: nan - acc: 0.6668

In [ ]:
# 클래스 이름과 인덱스 매핑 출력
print(train_generator.class_indices)

In [ ]:
# Accuracy 그래프
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Loss 그래프
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
